In [1]:
import numpy as np
import cv2
from sklearn.metrics import pairwise

In [2]:
background = None
box_top = 80
box_bottom = 300
box_right = 350
box_left = 550

accumulated_weight = 0.5

def calc_avg_wei(frame, accumulated_weight):
    global background
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [3]:
def segment(frame, threshold = 25):
    global background
    diff = cv2.absdiff(background.astype("uint8"),frame)
    res, thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    thresholded_temp = thresholded.copy()
    img, contours, cor = cv2.findContours(thresholded_temp,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0: 
        return None
    else:
        hand_segment = max(contours, key = cv2.contourArea)
        return (thresholded, hand_segment)

In [4]:
def count_fingers(thresholded, hand_segment):
    convex_hull = cv2.convexHull(hand_segment)
    
    top = tuple(convex_hull[convex_hull[:,:,1].argmin()][0])
    bottom = tuple(convex_hull[convex_hull[:,:,1].argmax()][0])
    right = tuple(convex_hull[convex_hull[:,:,0].argmin()][0])
    left = tuple(convex_hull[convex_hull[:,:,0].argmax()][0])
    
    cy = (bottom[1] + top[1]) // 2
    cx = (right[0] + left[0]) // 2
    
    distance = pairwise.euclidean_distances([(cx,cy)],Y = [left,right, top, bottom])[0]
    
    max_dis = distance.max()
    
    radius = int(max_dis*0.75)
    
    p = 2*np.pi*radius
    
    circular_roi = np.zeros(thresholded.shape[:2],dtype = 'uint8')
    
    cv2.circle(circular_roi,(cx,cy),radius,255,10)
    
    circular_roi = cv2.bitwise_and(thresholded,thresholded,mask = circular_roi)
    
    img, contours, cor = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    count = 0
     
    for i in contours:
        x,y,w,h = cv2.boundingRect(i)
        not_wrist = (((cy*0.35) + cy) > (y + h))
        not_noise = ((p*0.35) > i.shape[0])
        if not_wrist and not_noise: 
            count += 1
    
    if(count > 5): 
        return 5
    return count

In [ ]:
cap = cv2.VideoCapture(0)
cnt_frames = 0
global accumulated_weight
while True:
    ret, frame = cap.read()
    
    frame = cv2.flip(frame,1)
    
    frame_copy = frame.copy()
    
    roi = frame[box_top : box_bottom, box_right : box_left]
    
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.blur(gray,(15,15))
    if cnt_frames < 60:
        calc_avg_wei(gray,accumulated_weight)
        if cnt_frames <= 59:
            cv2.putText(frame_copy, 'Wait', (200,300),cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255),3)
         
    else: 
        hand_segment = segment(gray)
        if hand_segment is not None:
            threshold, hand = hand_segment
            cv2.drawContours(frame_copy, [hand + (box_right,box_top)],-1,(255,0,0),1)
            fingers = count_fingers(threshold, hand)
            cv2.putText(frame_copy,str(fingers), (50,70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255),3)
            cv2.imshow('fix', threshold)
            
    cv2.rectangle(frame_copy,(box_left,box_top),(box_right,box_bottom), (255,0,0), 5)
    cnt_frames += 1
    cv2.imshow("fingers", frame_copy)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows() 